In [1]:
import pandas as pd
import json
import random

from tqdm import tqdm
from collections import OrderedDict

In [2]:
#subjects_root_path = 'data/root'
train_data_path = 'data/root/train'
json_data_path = 'data/json'

In [3]:
""" subjects = []
for subject_dir in tqdm(os.listdir(subjects_root_path), desc='Iterating over subjects'):
    #print(subject_dir)
    subjects.append(int(subject_dir))
print(len(subjects)) """

" subjects = []\nfor subject_dir in tqdm(os.listdir(subjects_root_path), desc='Iterating over subjects'):\n    #print(subject_dir)\n    subjects.append(int(subject_dir))\nprint(len(subjects)) "

In [4]:
#print(len(subjects))

In [5]:
""" for index, row in df.iterrows():
    print(row['ICUSTAY_ID'], row['CHARTTIME'], row['ITEMID'], row['VALUE'])  """

" for index, row in df.iterrows():\n    print(row['ICUSTAY_ID'], row['CHARTTIME'], row['ITEMID'], row['VALUE'])  "

In [19]:
names = pd.read_csv('data/name.csv')
names = [i[0] for i in names.values.tolist()]
g_map = {1: '女', 2: '男', 3: None, 0: None}
e_map = {1: 'ASIAN',
         2: 'BLACK',
         3: 'SOUTH AMERICAN',
         4: 'WHITE',
         0: 'OTHER'
         }
try :
    os.makedirs(json_data_path)
except:
    pass

for subject in tqdm(os.listdir(train_data_path), desc='Iterating over subjects'):
     subjects_dict = OrderedDict()
     patient_dir = os.path.join(train_data_path, subject) 
     # 只读取某人第一次入住ICU的信息
     static_info = pd.read_csv(os.path.join(patient_dir, 'episode1.csv'))
     static_info = static_info.where(static_info.notnull(), None)
     icustay_id = int(static_info['Icustay'][0])
     
     events_info = pd.read_csv(os.path.join(patient_dir, 'events.csv'))
     #print(events_info['ICUSTAY_ID'])
     events_info['ICUSTAY_ID'] = events_info['ICUSTAY_ID'].fillna(-1).astype(int)
     #print(events_info['ICUSTAY_ID'])
     mask = events_info['ICUSTAY_ID']== icustay_id
     events_info = events_info.loc[mask] 
     events_info = events_info.where(events_info.notnull(), None)   
     
     patient_id = events_info['SUBJECT_ID'].iloc[0]
     name = random.choice(names)
     age = static_info['Age'][0]
     weight = static_info['Weight'][0]
     height = static_info['Height'][0] 
     gender = g_map[int(static_info['Gender'])]
     ethnicity = e_map[int(static_info['Ethnicity'])]
     subjects_dict['id'] = float(patient_id)
     subjects_dict['name'] = name
     subjects_dict['age'] = float(age)
     subjects_dict['weight'] = weight
     subjects_dict['height'] = height
     subjects_dict['gender'] = gender
     subjects_dict['ethnicity'] = ethnicity

     chartevents = []
     for index, row in events_info.iterrows():
         chart_dict = OrderedDict()
         #print(row['ICUSTAY_ID'], row['CHARTTIME'], row['ITEMID'], row['VALUE']) 
         charttime = row['CHARTTIME']
         itemid = row['ITEMID']
         value = row['VALUE']
         valueuom = row['VALUEUOM']
         chart_dict['charttime'] = charttime
         chart_dict['itemid'] = float(itemid)
         chart_dict['value'] = value
         chart_dict['valueuom'] = valueuom
         chartevents.append(chart_dict)
     subjects_dict['chartevents'] = chartevents
     subject_json_path  = os.path.join(json_data_path, '{}_.json'.format(subject))
     #subjects_dict = json.dumps(subjects_dict) 
     with open(subject_json_path, 'w') as jf:
         #jf.write(subjects_dict)
         json.dump(subjects_dict, jf, indent=4)


Iterating over subjects:   2%|▏         | 432/23658 [02:31<1:25:36,  4.52it/s]

FileNotFoundError: [Errno 2] File b'data/root/train/11242/episode1.csv' does not exist: b'data/root/train/11242/episode1.csv'